# The NAND<< Programming language

_Version: 0.2_

The NAND<< programming language was designed to accompany the upcoming book ["Introduction to Theoretical Computer Science"](http://introtcs.org). This is an appendix to this book, which is also available online as a Jupyter notebook in the  [boazbk/nandnotebooks](https://github.com/boazbk/nandnotebooks) on Github. 

The NAND++ programming language is defined in [Chapter 7: "Equivalent models of computation"](https://introtcs.org/public/lec_07_other_models.html)

Our models of computation can be thought of as follows:


* NAND language: _non uniform_, _finite_ model - equivalent to _Boolean circuits_.

* NAND++ language: _uniform_ model with _sequential_ or _oblivious_ (depending on the flavor) memory access - equivalent to _Turing machines_.

* NAND<< language: _uniform model_ with _random access memory_ - equivalent to _RAM machines_ and (if one does not care about polynomial factors) to Turing machines as well.

This notebook will eventually contain code to evaluate NAND<< programs, as well as to transform NAND<< code to NAND++ code. At the moment however we just have the specification of the language.

## The NAND<< programming language

The NAND<< (pronounced "NAND shift") programming language allows _indirection_, hence using variables  as  _pointer_ or _index_ variables.
Unlike the case of NAND++ vs NAND, NAND<< cannot compute functions that NAND++ can not (and indeed any NAND<< program can be "compiled to a NAND++ program) but it can be polynomially faster.

The main features of NAND<< are the following:

* Variables can hold values that are _arbitrary non negative integers_, rather than just zero or one.

* We use the convention that a variable whose name starts with a capital letter is an _array_ and a variable whose name starts with a lowercase letter is a _scalar_ variable.

* If `Foo` is an array and `bar` is a scalar, then `Foo[bar]` denotes the integer stored in the `bar`-th location of the array `Foo`

### NAND<< operations

Unlike our previous programming languages, NAND<< is not as minimalistic and contains a larger number of operations, many of which are redundant, in the sense that they can be implemented using other operations. The one component NAND<< does _not_ contain (though it can be of course implemented as syntatic sugar) is function calls.  This is because we want to maintain the invariant that an execution of a single line of NAND<< corresponds to a single computational step.


The NAND<< programming language allows the following operations:

* `foo = bar` (assignment)
* `foo = bar  + baz` (addition)
* `foo = bar - baz` (subtraction)
* `foo = bar >> baz` (right shift: $foo \leftarrow \lfloor bar 2^{-baz} \rfloor$)
* `foo = bar << baz` (left shift: $foo \leftarrow bar 2^{baz}$)
* `foo = bar % baz`  (modular reduction)
* `foo = bar * baz` (multiplication)
* `foo = bar / baz` (integer division: $foo \leftarrow \lfloor \tfrac{bar}{baz} \rfloor$)
* `foo = BITAND(bar,baz)` (bitwise AND)
* `foo = BITXOR(bar,baz)` (bitwise XOR)
* `foo = bar > baz` (greater than)
* `foo = bar < baz` (smaller than)
* `foo = EQUAL(bar,baz)` (equality)
* `foo = BOOL(bar)` (booleanize: `foo` gets $0$ if bar equals $0$ and gets $1$ otherwise)

We also use the following Boolean opertions (which apply `BOOL` implicitly to their input):

* `foo = NAND(bar,baz)`
* `foo = OR(bar,baz)`
* `foo = AND(bar,baz)`
* `foo = NOT(bar)`

In each one of the above `foo` , `bar` and `baz` denotes variables that are either of the form `scalarvar` or `Arrayvar[scalarvar]` (that is either a scalar variable or an array variable at a location indexed by a scalar variable).

We also have the following __control flow__ operations:

* `if foo: ...code... endif` performs `...code...` (which is a sequence of lines that starts with a newline) if  `foo` is nonzero.

* `while foo: ...code... endwhile` performs `...code...` as long as `foo` is nonzero.

* `do: ...code... until foo` performs `...code...` and if `foo` is zero then it goes back and does it again until `foo` becomes nonzero.



__Invariant:__ Each scalar variable or an element of an array variable in NAND<< can only hold an integer that ranges between $0$ and $t+1$ where $t$ is the number of lines of code (that is computational steps) that have been executed so far.  Another way to think about it as that we initialize $t=1$ in the beginning of the execution, and every time we execute a line of code, we increment $t$ by one. 

__Overflow and underflow:__ In all the operatoins above, they would have resulted in assigning a value to `foo` that is smaller than zero, then we assign zero instead. If they would have resulted in assigning a value to `foo` that is larger than $t+1$, then we assign $t+1$ instead. Note that one can ensure that the latter case does not happen by prefacing the operation with a loop that runs for at least $T$ times, where $T$ is an uppper bound on the value that we'll need. For this reason, this overflow restriction is immaterial when discussing issues of _computability_ and only makes a difference when one want to measure _running time_.

__Zero default:__ Just like NAND and NAND++, all variables that have not been assigned a value are assigned zero.

__No `loop` variable:__ Since NAND<< contains the `while` and `do` loops, we do not need an explicit `loop` variable. We can imitate the same semantics by simply using

```
do:
  ...program code...
  notloop = NOT(loop)
until 
```

### Ignore in first read: utility code for NAND++

We use some utility code, which you can safely ignore in first read, to allow us to write NAND++ code in Python

In [1]:
# utility code 
%run "NAND programming language.ipynb"
from IPython.display import clear_output
clear_output()

In [2]:
# Ignore this utility function in first and even second and third read
import inspect
import ast
import astor

def noop(f):
    return f;

def runwithstate(f):
    """Modify a function f to take and return an argument state and make all names relative to state."""
    tree = ast.parse(inspect.getsource(f))
    tmp = ast.parse("def _temp(state):\n    pass\n").body[0]
    args = tmp.args
    name = tmp.name
    tree.body[0].args = args
    tree.body[0].name = name
    tree.body[0].decorator_list = []
    

    class AddState(ast.NodeTransformer):
        def visit_Name(self, node: ast.Name):
            if node.id == "enandpp": return ast.Name(id="noop", ctx=Load())
            new_node = ast.Attribute(ast.copy_location(ast.Name('state', ast.Load()), node), node.id,
                                     ast.copy_location(ast.Load(), node))
            return ast.copy_location(new_node, node)
        
    tree = AddState().visit(tree)
    tree.body[0].body = tree.body[0].body + [ast.parse("return state")]
    tree = ast.fix_missing_locations(tree)
    src = astor.to_source(tree)
    # print(src)
    exec(src,globals())
    _temp.original_func = f
    return _temp

    
def enandpp(f):
    g = runwithstate(f)
    def _temp1(X):
        nonlocal g
        return ENANDPPEVAL(g,X)
    _temp1.original_func = f
    _temp1.transformed_func = g
    return _temp1

In [3]:
# ignore utility class in first and even second or third read

from  collections import defaultdict
class NANDPPstate:
    """State of a NAND++ program."""
    
    def __init__(self):
        self.scalars = defaultdict(int)
        self.arrays  = defaultdict(lambda: defaultdict(int))
        # eventually should make self.i non-negative integer type
        
    def __getattr__(self,var):
        g =  globals()
        if var in g and callable(g[var]): return g[var]
        if var[0].isupper():
            return self.arrays[var]
        else:
            return self.scalars[var]

In [4]:
def ENANDPPEVAL(f,X):
    """Evaluate an enhanced NAND++ function on input X"""
    s = NANDPPstate()
    for i in range(len(X)):
        s.X[i] = X[i]
        s.Xvalid[i] = 1
    while True:
        s = f(s)
        if not s.loop: break
    res = []
    i = 0
    while s.Yvalid[i]: 
        res += [s.Y[i]]
        i+= 1
    return res

In [5]:
def rreplace(s, old, new, occurrence=1): # from stackoverflow
    li = s.rsplit(old, occurrence)
    return new.join(li)

    
        
def ENANDPPcode(P):
    """Return ENAND++ code of given function"""
    
    code = ''
    counter = 0
    
    class CodeENANDPPcounter:
        def __init__(self,name="i"): 
            self.name = name
        
        def __iadd__(self,var):
            nonlocal code
            code += f'\ni += {var}'
            return self
        
        def __isub__(self,var):
            nonlocal code
            code += f'\ni -= {var}'
            return self
        
        def __str__(self): return self.name
    
    class CodeNANDPPstate:
    
    
        def __getattribute__(self,var):
            # print(f"getting {var}")
            if var=='i': return CodeENANDPPcounter()
            g =  globals()
            if var in g and callable(g[var]): return g[var]
            if var[0].isupper():  
                class Temp:
                    def __getitem__(self,k):  return f"{var}[{str(k)}]"
                    def __setitem__(s,k,v): setattr(self,f"{var}[{str(k)}]",v)            
                return Temp()
            return var
    
        def __init__(self):
            pass
    
        def __setattr__(self,var,val):
            nonlocal code
            if var=='i': return
            if code.find(val)==-1:
                code += f'\n{var} = {val}'
            else:
                code = rreplace(code,val,var)
    
    s = CodeNANDPPstate()
    
    def myNAND(a,b):
        nonlocal code, counter
        var = f'temp_{counter}'
        counter += 1
        code += f'\n{var} = NAND({a},{b})'
        return var
    
        
    
    
    
    s = runwith(lambda : P.transformed_func(s),"NAND",myNAND) 
    
    return code

In [ ]:
@enandpp
def inc():
    carry = IF(started,carry,one(started))
    started = one(started)
    Y[i] = XOR(X[i],carry)
    carry = AND(X[i],carry)
    Yvalid[i] = one(started)
    loop = COPY(Xvalid[i])
    i += loop

In [ ]:
def index():
    """Generator for the values of i in the NAND++ sequence"""
    i = 0
    last = 0
    direction  = 1
    while True:
        yield i
        i += direction
        if i> last: 
            direction = -1
            last = i
        if i==0: direction = +1
            
a = index()
[next(a) for i in range(20)]

In [ ]:
def NANDPPEVAL(f,X):
    """Evaluate a NAND++ function on input X"""
    s = NANDPPstate() # intialize state
    
    # copy input:
    for i in range(len(X)): 
        s.X[i] = X[i]
        s.Xvalid[i] = 1
        
    # main loop:
    for  i in index(): 
        s.i = i
        s = f(s)
        if not s.loop: break
    
    # copy output:
    res = [] 
    i = 0
    while s.Yvalid[i]: 
        res += [s.Y[i]]
        i+= 1
    return res



def nandpp(f):
    """Modify python code to obtain NAND++ program"""
    g = runwithstate(f)
    def _temp1(X):
        return NANDPPEVAL(g,X)
    _temp1.original_func = f
    _temp1.transformed_func = g
    return _temp1

In [ ]:
@nandpp
def inc():
    carry = IF(started,carry,one(started))
    started = one(started)
    Y[i] = IF(Visited[i],Y[i],XOR(X[i],carry))
    Visited[i] = one(started)
    carry = AND(X[i],carry)
    Yvalid[i] = one(started)
    loop = Xvalid[i]

And here is the "vanilla NAND++" version of XOR:

In [ ]:
@nandpp
def vuXOR():
    Yvalid[0] = one(X[0])
    Y[0] = IF(Visited[i],Y[0],XOR(X[i],Y[0]))
    Visited[i] = one(X[0])
    loop = Xvalid[i]

In [ ]:
vuXOR([1,0,0,1,0,1,1])

In [ ]:
def NANDPPcode(P):
    """Return NAND++ code of given function"""
    
    code = ''
    counter = 0
    
    
    class CodeNANDPPstate:
    
    
        def __getattribute__(self,var):
            # print(f"getting {var}")
            g =  globals()
            if var in g and callable(g[var]): return g[var]
            if var[0].isupper():  
                class Temp:
                    def __getitem__(self,k):  return var+"[i]"
                    def __setitem__(s,k,v):   
                        setattr(self,var+"[i]",v)            
                return Temp()
            return var
    
        def __init__(self):
            pass
    
        def __setattr__(self,var,val):
            nonlocal code
            # print(f"setting {var} to {val}")
            if code.find(val)==-1:
                code += f'\n{var} = {val}'
            else:
                code = rreplace(code,val,var)
    
    s = CodeNANDPPstate()
    
    def myNAND(a,b):
        nonlocal code, counter
        var = f'temp_{counter}'
        counter += 1
        code += f'\n{var} = NAND({a},{b})'
        return var
    
        
    
    
    
    s = runwith(lambda : P.transformed_func(s),"NAND",myNAND) 
    
    return code


# utility code - replace string from right, taken from stackoverflow
def rreplace(s, old, new, occurrence=1): 
    li = s.rsplit(old, occurrence)
    return new.join(li)



In [ ]:
print(NANDPPcode(inc))